In [ ]:
!cd ../yolov3 && git checkout .

In [ ]:
import sys
sys.path.append("../yolov3")

In [ ]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cpu')

In [ ]:
from models.yolo import Model
model = Model('../yolov3/models/yolov3-tiny.yaml', ch=3, nc=80, anchors=3).to(device)
for k, v in model.named_parameters():
    v.requires_grad = True  

Overriding model.yaml anchors with anchors=3

                 from  n    params  module                                  arguments                     
  0                -1  1       464  models.common.Conv                      [3, 16, 3, 1]                 
  1                -1  1         0  torch.nn.modules.pooling.MaxPool2d      [2, 2, 0]                     
  2                -1  1      4672  models.common.Conv                      [16, 32, 3, 1]                
  3                -1  1         0  torch.nn.modules.pooling.MaxPool2d      [2, 2, 0]                     
  4                -1  1     18560  models.common.Conv                      [32, 64, 3, 1]                
  5                -1  1         0  torch.nn.modules.pooling.MaxPool2d      [2, 2, 0]                     
  6                -1  1     73984  models.common.Conv                      [64, 128, 3, 1]               
  7                -1  1         0  torch.nn.modules.pooling.MaxPool2d      [2, 2, 0]             

In [ ]:
import torch.nn as nn
from torch.optim import SGD, Adam, lr_scheduler
g0, g1, g2 = [], [], [] 
for v in model.modules():
    if hasattr(v, 'bias') and isinstance(v.bias, nn.Parameter):
        g2.append(v.bias)
    if isinstance(v, nn.BatchNorm2d): 
        g0.append(v.weight)
    elif hasattr(v, 'weight') and isinstance(v.weight, nn.Parameter): 
        g1.append(v.weight)
#optimizer = Adam(g0, lr=0.01, betas=(0.937, 0.999))
optimizer = SGD(g0, lr=0.01, momentum=0.937, nesterov=True)
optimizer.add_param_group({'params': g1, 'weight_decay': 0.0005}) 
optimizer.add_param_group({'params': g2}) 

In [ ]:
import yaml
from utils.datasets import create_dataloader
f = open("/work/yolov3/data/hyps/hyp.scratch.yaml", errors='ignore')
hyp = yaml.safe_load(f)
train_loader, dataset = create_dataloader("/work/datasets/coco128/images/train2017", 640, 3, 32, False,
                                              hyp=hyp, augment=True, cache='ram', rect=False, rank=-1,
                                              workers=8, image_weights=False, quad=False,
                                              prefix='train: ', shuffle=True)

train: Scanning '/work/datasets/coco128/labels/train2017' images and labels...128 found, 0 missing, 2 empty, 0 corrupted: 100%|██████████| 128/128 [00:00<00:00, 536.12it/s]
train: New cache created: /work/datasets/coco128/labels/train2017.cache
train: Caching images (0.1GB ram): 100%|██████████| 128/128 [00:00<00:00, 195.21it/s]


In [ ]:
model.nc = 80  
model.hyp = hyp
model.names = [ 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
         'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
         'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
         'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
         'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
         'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
         'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
         'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
         'hair drier', 'toothbrush' ]

In [ ]:
from torch.cuda import amp
amp.autocast(enabled=False)
scaler = amp.GradScaler(enabled=False)

/shared-libs/python3.9/py/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [ ]:
from utils.loss import ComputeLoss
compute_loss = ComputeLoss(model)

In [ ]:
import tqdm
from utils.torch_utils import de_parallel
from copy import deepcopy
for epoch in range(2):
    model.train()
    optimizer.zero_grad()
    for i, (imgs, targets, paths, _) in enumerate(tqdm.tqdm(train_loader)):
        imgs = imgs.float() / 255
        with amp.autocast(enabled=True):
            pred = model(imgs)  # forward
            loss, loss_items = compute_loss(pred, targets)
        scaler.scale(loss).backward()
        scaler.step(optimizer)  # optimizer.step
        scaler.update()
        optimizer.zero_grad()
ckpt = {'epoch': 200,
        'best_fitness': 0,
        'model': deepcopy(de_parallel(model)).half(),
        'ema': None,
        'updates': 0,
        'optimizer': optimizer.state_dict(),
        'wandb_id': None,
        'date': None}
torch.save(ckpt, "/datasets/last.pt")

100%|██████████| 43/43 [04:50<00:00,  6.75s/it]


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4c27263b-a773-4b36-bc5c-6cb959f3ea6f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>